In [1]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)

openai = OpenAI()

In [3]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
push("HEY!!")

Push: HEY!!


In [ ]:
# Tool function for recording user details
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
# Tool function an LLM uses when it doesn't know the answer to a user's question
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
# this information is going to be sent to OpenAI LLM
# we are telling LLM : you have the ability to do this, tell me If you want me to do it for you
# it's a packaged way of describing what this function does in JSON format so an LLM can then decide in its response whether or not -->
# it wants to actually call this tool

# there is a lot of JSON in the training data

record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [8]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [9]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [10]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [ ]:
# This function can take a list of tool calls, and run them
# this func will handle what happens when the LLM response "Yes I do want to run this tool, pls run it and provide me with output"

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        # tool_calls:
        # [ChatCompletionMessageToolCall(id='call_zjWM3jjS7uXpo4MBfCcQgPws', 
        # function=Function(arguments='{"question":"What is Daniel Katzor\'s favorite movie genre?"}', 
        # name='record_unknown_question'), type='function')]
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        # globals gives us a dictionary which I can use to lookup any function which is in the global scope
        tool = globals().get(tool_name)
        
        result = tool(**arguments) if tool else {}
        # this is the result:
        # {'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'call_d2i2kADu9NBly2ROXNfFv1Qf'}
        results.append({"role": "tool",
                        "content": json.dumps(result),
                        "tool_call_id": tool_call.id}) # id='call_zjWM3jjS7uXpo4MBfCcQgPws'
        
    return results

In [ ]:
# load up the linkedin profile and a summary information provided for an LLM

reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Your name"

In [18]:
# we repeat the fact that LLM agent can use these tools
# in theory this isn't needed because the JSON we used to described the tool already gives this kind of context
# it never hurts to be repetitive in prompting, it will increase the probability that the model will generate tokens ->
# consistent with my objective, we are biasing the model to output tokens consistent with the objective
# it's good to remind ourselves that LLM is something that generates the most likely next token


system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    # for printing:
    message_user = message
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        # this tells us whether the LLM is done or wants to call a tool
        # finish_reason='stop', 'tool_calls'
        finish_reason = response.choices[0].finish_reason

        print(f"Printing 'finish_reason = response.choices[0].finish_reason': \n {finish_reason}")
        
        # If the LLM wants to call a tool, we do that
         
        if finish_reason=="tool_calls":
            print(f"src: chat / printing message typed by a user after finish reason is tool: \n {message}")
            message = response.choices[0].message
            print(f"src: chat / printing message as response from the LLM after finish reason is tools > message = response.choices[0].message: \n {message}")
            # now the message should contain: ChatCompletionMessage(content='text returned by LLM',
            #                    refusal=None, 
            #                    role='assistant', 
            #                    annotations=[], 
            #                    audio=None, 
            #                    function_call=None, 
            #                    tool_calls=None)

            # message now represents an object response.choices[0].message, so I can get an attribute: message.audio. message.tool_calls
            tool_calls = message.tool_calls
            print(f"From chat function printing 'tool_calls = message.tool_calls' \n {tool_calls}")
            results = handle_tool_calls(tool_calls)

            print(f"src: chat / results = handle_tool_calls(tool_calls) : \n {results}")

            # we append messages with the message and the results and then we loop back and call LLM again until it doesn't want to use tool ->
            # in that case the else statemnt is executed and done is set to True >> the chat function returns just the content from LLM to the user
            # the LLM could want to use the tool for recording the contact details so there's a possibility of using another tool => use of the while loop
            # what it wanted to do / it's just a message
            messages.append(message)
            print(f"src: chat / print messages after messages.append(message): \n{messages}")
            reply = response.choices[0].message.content
            print(f"src: chat / Printing message.content: \n{reply}")
            # the result of doing it
            # {'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'call_d2i2kADu9NBly2ROXNfFv1Qf'}
            messages.extend(results)
            print(f"src: chat / print messages after messages.extend(results): \n{messages}")
        else:
            done = True
            print(f"src: chat / printing message typed by a user when tool is not used: \n {message_user}")
    print(f"src: chat / Printing message.content from LLM: \n{response.choices[0].message.content}")
    return response.choices[0].message.content 
    # what is response.choices[0].message.content? , is it content from role: assistant?
    # yes, it's the response to the user
    # is it {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options' = None}

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## Build in the Evaluator

In [1]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [20]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [21]:
# it takes the reply of the original message
# message - original message which the reply was replying to


def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [22]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [23]:
# this function will take again reply and original message + history
# it will return one of these objects we defined here: class Evaluation(BaseModel)
# it uses a technique called Structured outputs which is a way that I can require an LLM to response in a form of an object like this
# it's JSON behind the scenes

def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    # now comes the way you pull the API to use structured outputs (.parse)
    # we pass model, messages and specify an object we want to be populated -->
    # I'm giving it a schema, a pydantic object and say >> I want you to respond with this object
    # it's going to actually respond with JSON and the client library is going to take that JSON and use it to populate the object
    # so it's an impression that we are getting a code from LLM
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    # so it returns an instance of the Evaluation populated with the response from the LLM
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a certification related to AI?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [24]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):

    if "certification" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    done = False

    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        if response.choices[0].finish_reason=="tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = handle_tool_call(tool_calls)
                messages.append(message)
                messages.extend(results)
        else:
            done = True
    
    # evaluator block
    reply = response.choices[0].message.content
    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print(f"Passed evaluation - returning reply: \n{reply}")
    else:
        print(f"Failed evaluation - retrying - failed reply: \n{reply}")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)      
    return reply

In [29]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
